In [2]:
import numpy as np
import pymc as pm
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import arviz as az
from lets_plot import *
LetsPlot.setup_html()  # For Jupyter/Colab environment

In [4]:

# Set random seed for reproducibility
np.random.seed(42)

# Generate a high-dimensional synthetic classification dataset
# 5 informative features, 5 redundant features
X, y = make_classification(n_samples=200, n_features=10, n_informative=5,
                          n_redundant=5, n_classes=2,
                          class_sep=1.5, random_state=42)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to DataFrames for easier handling
feature_names = [f'X{i+1}' for i in range(X.shape[1])]
train_df = pd.DataFrame(X_train_scaled, columns=feature_names)
train_df['y'] = y_train
test_df = pd.DataFrame(X_test_scaled, columns=feature_names)
test_df['y'] = y_test

# Function to create pairplot for visualizing high-dimensional data
def create_pairplot(data, features, target, n_features=3):
    """Create a pairplot for the first n_features"""
    plot_data = data.copy()
    selected_features = features[:n_features]

    # Melt the dataframe for plotting
    melted_data = []
    for i, feat1 in enumerate(selected_features):
        for j, feat2 in enumerate(selected_features):
            if i < j:  # Only use upper triangle
                for idx, row in plot_data.iterrows():
                    melted_data.append({
                        'x_feature': feat1,
                        'y_feature': feat2,
                        'x_value': row[feat1],
                        'y_value': row[feat2],
                        'class': row[target]
                    })

    return pd.DataFrame(melted_data)

# Create pairplot data
pairplot_data = create_pairplot(train_df, feature_names, 'y', n_features=3)

# Plot pairwise relationships
if not pairplot_data.empty:
    p_pair = ggplot(pairplot_data, aes(x='x_value', y='y_value', color='class')) + \
        geom_point(alpha=0.7) + \
        facet_grid(y='y_feature', x='x_feature', scales='free') + \
        scale_color_hue(name="Class") + \
        theme_light() + \
        theme(axis_text=element_text(size=8))

    p_pair.show()

In [ ]:

# Define Bayesian logistic regression model using PyMC
with pm.Model() as logistic_model:
    # Priors for unknown model parameters
    # Using weakly informative priors
    alpha = pm.Normal('alpha', mu=0, sigma=3)  # Intercept

    # Coefficients for each feature
    betas = pm.Normal('betas', mu=0, sigma=3, shape=X_train_scaled.shape[1])

    # Expected value of outcome
    mu = alpha + pm.math.dot(X_train_scaled, betas)

    # Likelihood (sampling distribution) of observations
    y_obs = pm.Bernoulli('y_obs', logit_p=mu, observed=y_train)

    # Sample from the posterior - using NUTS sampler
    # Using fewer samples to speed up computation for high-dimensional data
    trace = pm.sample(1000, tune=500, return_inferencedata=True,
                      cores=1, random_state=42)

# Extract posterior samples for the parameters
alpha_samples = trace.posterior['alpha'].values.flatten()
beta_samples = trace.posterior['betas'].values.reshape(-1, X_train_scaled.shape[1])

# Create a DataFrame with the beta samples for visualization
beta_sample_data = []
for i in range(len(feature_names)):
    for sample in beta_samples[:100]:  # Use the first 100 samples for visualization
        beta_sample_data.append({
            'feature': feature_names[i],
            'coefficient': sample[i]
        })

beta_df = pd.DataFrame(beta_sample_data)

In [9]:

# Plot the posterior distributions of coefficients
p_coef = ggplot(beta_df, aes(x='coefficient', fill='feature')) + \
    geom_density(alpha=0.5) + \
    facet_wrap('feature', scales='free_y') + \
    ggtitle("Posterior Distributions of Feature Coefficients") + \
    theme_light() + \
    theme(axis_text=element_text(size=8), legend_position='none')

p_coef.show()

In [11]:

# Function to make predictions using posterior samples
def predict_proba(X, alpha, betas):
    logits = alpha + np.dot(X, betas.T)
    return 1.0 / (1.0 + np.exp(-logits))

def predict_class(X, alpha, betas, threshold=0.5):
    probs = predict_proba(X, alpha, betas)
    return (probs.mean(axis=1) > threshold).astype(int)

# Make predictions on test set
test_probs = predict_proba(X_test_scaled, alpha_samples, beta_samples)
test_pred = predict_class(X_test_scaled, alpha_samples, beta_samples)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, test_pred)
roc_auc = roc_auc_score(y_test, test_probs.mean(axis=1))

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, test_pred))

# Visualize prediction uncertainty
test_pred_samples = (test_probs > 0.5).astype(int)
test_pred_variance = test_pred_samples.var(axis=1)

# Create DataFrame for prediction uncertainty visualization
uncertainty_df = pd.DataFrame({
    'True_Class': y_test,
    'Pred_Class': test_pred,
    'Uncertainty': test_pred_variance,
    'Correct': y_test == test_pred
})

# Plot prediction uncertainty
p_uncertainty = ggplot(uncertainty_df, aes(x='Uncertainty', fill='Correct')) + \
    geom_histogram(bins=20, alpha=0.7, position='identity') + \
    scale_fill_manual(values=['red', 'green'], name='Prediction',
                     labels=['Incorrect', 'Correct']) + \
    ggtitle("Prediction Uncertainty vs Correctness") + \
    theme_light()

p_uncertainty.show()

Test Accuracy: 0.8000
Test ROC-AUC: 0.8348

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.71      0.80        34
           1       0.71      0.92      0.80        26

    accuracy                           0.80        60
   macro avg       0.81      0.81      0.80        60
weighted avg       0.83      0.80      0.80        60



In [15]:

# Visualize decision boundaries in 2D projection
# For clarity, we'll use the first two principal components
from sklearn.decomposition import PCA

# Apply PCA to reduce to 2 dimensions
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Create a meshgrid for visualization
h = 0.1  # Step size
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
grid_points = np.c_[xx.ravel(), yy.ravel()]

# We need to project these grid points back to original feature space
# Since PCA is not invertible for dimensionality reduction,
# we'll use a simpler approach: train a model directly on the PCA space

# Define a simpler Bayesian logistic regression model on PCA space
with pm.Model() as pca_model:
    # Priors for parameters in PCA space
    alpha_pca = pm.Normal('alpha_pca', mu=0, sigma=3)
    betas_pca = pm.Normal('betas_pca', mu=0, sigma=3, shape=2)

    # Expected value of outcome
    mu_pca = alpha_pca + pm.math.dot(X_train_pca, betas_pca)

    # Likelihood of observations
    y_obs_pca = pm.Bernoulli('y_obs_pca', logit_p=mu_pca, observed=y_train)

    # Sample from the posterior
    trace_pca = pm.sample(1000, tune=500, return_inferencedata=True,
                         cores=1, random_seed=42)

# Extract posterior samples for PCA model
alpha_pca_samples = trace_pca.posterior['alpha_pca'].values.flatten()
beta_pca_samples = trace_pca.posterior['betas_pca'].values.reshape(-1, 2)

# Function to make predictions in PCA space
def predict_proba_pca(X, alpha, betas):
    logits = alpha + np.dot(X, betas.T)
    return 1.0 / (1.0 + np.exp(-logits))

# Get probability predictions for the grid
grid_probs = predict_proba_pca(grid_points, alpha_pca_samples, beta_pca_samples)
grid_mean_probs = grid_probs.mean(axis=1).reshape(xx.shape)

# Create a DataFrame for the decision boundary plot
boundary_df = pd.DataFrame({
    'x': xx.ravel(),
    'y': yy.ravel(),
    'prob': grid_mean_probs.ravel()
})

# Create DataFrames for train and test points
train_pca_df = pd.DataFrame({
    'x': X_train_pca[:, 0],
    'y': X_train_pca[:, 1],
    'class': y_train
})

test_pca_df = pd.DataFrame({
    'x': X_test_pca[:, 0],
    'y': X_test_pca[:, 1],
    'class': y_test,
    'pred': test_pred
})

# Plot decision boundary and points
p_boundary = ggplot() + \
    geom_tile(data=boundary_df, mapping=aes(x='x', y='y', fill='prob'), alpha=0.7) + \
    scale_fill_gradient2(low='blue', mid='white', high='red', midpoint=0.5) + \
    geom_point(data=train_pca_df, mapping=aes(x='x', y='y', color='class', shape='class'),
              size=3, alpha=0.7) + \
    scale_color_hue(name="True Class") + \
    ggtitle("Bayesian Decision Boundary (PCA Projection)") + \
    labs(x="Principal Component 1", y="Principal Component 2") + \
    theme_light()

p_boundary.show()

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [alpha_pca, betas_pca]


Output()

Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 1 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


In [19]:

# Plot test set predictions
p_test = ggplot(test_pca_df, aes(x='x', y='y', color='class', shape='pred')) + \
    geom_point(size=3, alpha=0.8) + \
    scale_color_hue(name="True Class") + \
    ggtitle("Test Set Predictions in PCA Space") + \
    labs(x="Principal Component 1", y="Principal Component 2") + \
    theme_light()

p_test.show()

# Visualize individual decision boundaries from posterior samples
# Create multiple decision boundaries from different posterior samples
boundary_samples = []
for i in range(10):  # Use 10 random samples
    idx = np.random.randint(0, len(alpha_pca_samples))
    a, b = alpha_pca_samples[idx], beta_pca_samples[idx]

    # Calculate the decision boundary line
    # At the boundary, logit = 0 = a + b1*x + b2*y
    # Solving for y: y = (-a - b1*x) / b2
    if b[1] != 0:  # Avoid division by zero
        slope = -b[0] / b[1]
        intercept = -a / b[1]

        x_vals = np.array([x_min, x_max])
        y_vals = slope * x_vals + intercept

        for j in range(len(x_vals)):
            boundary_samples.append({
                'x': x_vals[j],
                'y': y_vals[j],
                'sample_id': i
            })

boundary_lines_df = pd.DataFrame(boundary_samples)

# Plot the decision boundaries from different posterior samples
p_lines = ggplot() + \
    geom_point(data=train_pca_df, mapping=aes(x='x', y='y', color='class'),
              size=2, alpha=0.5) + \
    geom_line(data=boundary_lines_df, mapping=aes(x='x', y='y', group='sample_id'),
             color='black', alpha=0.3) + \
    scale_color_hue(name="Class") + \
    ggtitle("Multiple Linear Decision Boundaries from Posterior Samples") + \
    labs(x="Principal Component 1", y="Principal Component 2") + \
    theme_light() + \
    xlim(x_min, x_max) + ylim(y_min, y_max)

p_lines.show()

# Display the average coefficients
avg_alpha = alpha_samples.mean()
avg_betas = beta_samples.mean(axis=0)

coef_summary = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': avg_betas,
    'Abs_Value': np.abs(avg_betas)
}).sort_values('Abs_Value', ascending=False)

print("\nAverage Model Coefficients (sorted by importance):")
print(coef_summary)

# Visualize overall coefficient magnitudes
p_importance = ggplot(coef_summary, aes(x='Feature', y='Coefficient')) + \
    geom_bar(stat='identity', fill='blue') + \
    coord_flip() + \
    ggtitle("Feature Coefficient Magnitudes") + \
    labs(x="Feature", y="Coefficient Value") + \
    theme_light()

p_importance.show()


Average Model Coefficients (sorted by importance):
  Feature  Coefficient  Abs_Value
0      X1    -1.098092   1.098092
1      X2    -0.898275   0.898275
3      X4     0.732662   0.732662
4      X5     0.715178   0.715178
8      X9    -0.573970   0.573970
6      X7    -0.526372   0.526372
9     X10     0.490288   0.490288
5      X6     0.361049   0.361049
7      X8     0.165541   0.165541
2      X3     0.097921   0.097921
